## 实现反向传播（Back Propagation）

本节我们将做两个独立示例，一个回归例子，一个分类例子。

为阐释如何使用 tf 反向传播，我们先加载必要的库并重置计算图。

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

### 创建图会话

In [2]:
sess = tf.Session()

### 回归示例
---

我们创建一个如下回归示例：输入数据为 100 个 normal 随机采样（mean: 1.0, stdev: 0.1）。目标值是 100 个 10.0

如此适配回归模型：x_data * A = target_values

理论上，我们知道 A 应该等于 10.0。

先创建数据和目标并 placeholder

In [4]:
x_vals = np.random.normal(1, 0.1, 100)
y_vals = np.repeat(10., 100)
x_data = tf.placeholder(shape=[1], dtype=tf.float32)
y_target = tf.placeholder(shape=[1], dtype=tf.float32)

现在创建用于计算图的变量，A

In [5]:
# 创建变量（一个模型变量=A）
A = tf.Variable(tf.random_normal(shape=[1]))

把模型操作加入图。下面只是把输入乘 A 已得到输出。

In [6]:
# 添加到计算图
my_output = tf.multiply(x_data, A)

接下来我们指定损失函数。这将允许 tf 知道如何改变模型变量。我们使用 L2 损失函数。注意：若要用 L1 损失函数，把 `tf.square()` 改为 `tf.abs()`

In [7]:
# 添加 L2 损失函数到计算图
loss = tf.square(my_output - y_target)

现在初始化变量，需要指出的是，这是使用随机标准正常数字在计算图上初始化变量A

In [8]:
# 初始化变量
init = tf.global_variables_initializer()
sess.run(init)

我们需要创建一个优化操作。这里我们使用标准的 `GradientDescentOptimizer()`，并告诉 tf 最小化损失。这里使用 0.02 的学习率，但可在这个值附近随心所欲的尝试，最后可看到学习曲线。然而，请注意，学习率过大将导致算法不会收敛。

In [9]:
# 创建优化器
my_opt = tf.train.GradientDescentOptimizer(0.02)
train_step = my_opt.minimize(loss)

### 运行回归图

接下来迭代运行计算图 100 次，没迭代 25 次打印 A 和 损失值。我们将看到 A 值越来越趋近 10，同时损失降低。

In [11]:
# 循环运行
for i in range(100):
    rand_index = np.random.choice(100)
    rand_x = [x_vals[rand_index]]
    rand_y = [y_vals[rand_index]]
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    if (i+1) % 25 == 0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)))
        print('Loss = ' + str(sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})))

Step #25 A = [ 9.92020321]
Loss = [ 1.03096735]
Step #50 A = [ 9.88349724]
Loss = [ 0.3415648]
Step #75 A = [ 9.77454758]
Loss = [ 0.04842026]
Step #100 A = [ 10.07893467]
Loss = [ 4.997334]


## 分类例子
---

分类例子，我们创建两个由不同的 normal 分发输入制作的 x-采样，Normal(mean=-1, sd=1) 和 Normal(mean=3, sd=1)。它们的每个目标值将会是 0 或 1。

该模型将适配二进制分类：if sigmoid(x+A) < 0.5 then 预测为 类 0 else 类 1.

理论上，我们知道 A 值应该在两个 means 中去负的平均值 -(mean1 + mean2) / 2。

重置计算图：

In [26]:
ops.reset_default_graph()

启动图会话

In [27]:
sess = tf.Session()

先生成将在计算图中使用的数据。注意 x_vals 是两个独立 normal 的组合，y_vals 是两个独立 constant 的组合（两个分类）。

我们还为模型创建了相关的 placeholder。

In [28]:
# 创建数据
x_vals = np.concatenate((np.random.normal(-1, 1, 50), np.random.normal(3, 1, 50)))
y_vals = np.concatenate((np.repeat(0., 50), np.repeat(1., 50)))
x_data = tf.placeholder(shape=[1], dtype=tf.float32)
y_target = tf.placeholder(shape=[1], dtype=tf.float32)

现在创建一个模型变量，用于分类。我们也设置初始化函数，一个随机 normal（正态分布），来获得远离预期理论值的平均值。

- 初始化值围绕在 10.0
- 理论上围绕 -1.0 

In [29]:
# 创建变量（模型参数 A）
A = tf.Variable(tf.random_normal(mean=10, shape=[1]))

现在添加模型操作到图上。这将把变量 A 添加到数据。注意 sigmoid() 没有在这个操作里，因为我们使用的损失函数已经包含它了。

In [30]:
# 添加操作到图
# 想创建操作 sigmoid(x + A)
# 注意，sigmoid() 已经在损失函数里了
my_output = tf.add(x_data, A)

# 现在我们必须添加另一维度到每个数据（批量大小为 1）
my_output_expanded = tf.expand_dims(my_output, 0)
y_target_expanded = tf.expand_dims(y_target, 0)

添加分类损失损失（交叉熵）

In [31]:
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=my_output_expanded, labels=y_target_expanded)

现在声明优化函数。这里使用标准梯度递减操作，学习率为 0.05

In [32]:
# 创建优化器
my_opt = tf.train.GradientDescentOptimizer(0.05)
train_step = my_opt.minimize(xentropy)

接下来创建初始化变量的操作并运行这个操作

In [33]:
# 初始化变量
init = tf.global_variables_initializer()
sess.run(init)

### 运行分类图

现在循环分类图并打印 A 和损失函数的值

In [34]:
# 循环图
for i in range(1400):
    rand_index = np.random.choice(100)
    rand_x = [x_vals[rand_index]]
    rand_y = [y_vals[rand_index]]
    
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    if (i+1) % 200 == 0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)))
        print('Loss = ' + str(sess.run(xentropy, feed_dict={x_data: rand_x, y_target: rand_y})))

Step #200 A = [ 4.88592863]
Loss = [[ 4.19602108]]
Step #400 A = [ 1.24687517]
Loss = [[ 1.25257277]]
Step #600 A = [-0.36788091]
Loss = [[ 0.22166912]]
Step #800 A = [-0.69072759]
Loss = [[ 0.01465659]]
Step #1000 A = [-0.91324085]
Loss = [[ 0.03224439]]
Step #1200 A = [-1.0617311]
Loss = [[ 0.0660689]]
Step #1400 A = [-0.9054684]
Loss = [[ 0.17460336]]


现在我们看一下预测结果：

In [35]:
# 预测评估
predictions = []
for i in range(len(x_vals)):
    x_val = [x_vals[i]]
    prediction = sess.run(tf.round(tf.sigmoid(my_output)), feed_dict={x_data: x_val})
    predictions.append(prediction)
    
accuracy = sum(x==y for x, y in zip(predictions, y_vals))/100.
print('Ending Accuracy = ' + str(np.round(accuracy, 2)))

Ending Accuracy = [ 0.98]
